In [7]:
import sys 

# Note: I do this so that I can import files across folders
# See this discussion: https://stackoverflow.com/questions/75990606/automatically-add-project-to-sys-path-in-vs-code-like-pycharm-spyder-do
sys.path.insert(1, '/Users/ryanstevens/Documents/github/torch_practice')     
sys.path.insert(0, '/Users/ryanstevens/Documents/github/torch_practice')     

In [ ]:
%%markdown

# Static Autoencoder Tutorial

At work, we've been starting to explore different approaches to anomaly detection. I'm creating a series of posts to clarify my understanding of autoencoders, and how to use them for anomaly detection. I am going to start

This post will focus on a very simple setup, a static autoencoder trying to identify anomalous images in the `MNIST` hand-written digit dataset. 

## Why do we need autoencoders? 

The key idea is lossless compression. Below shows the process of compressing and uncompressing a set of data points from $\mathbb{R}^{2} \to \mathbb{R}^{1}$ and back to $\mathbb{R}^{1} \to \mathbb{R}^{2}$. In the example, we have three sets of data points:
* Input: $\bold{x} \in \mathbb{R}^{2}$
* Compressed by map $g_{\theta_{z}}$: $\bold{z} \in \mathbb{R}^{1}$
* Reconstructed by map $f_{\theta_{x}}$: $\bold{\tilde{x}} \in \mathbb{R}^{2}$

![]('/Users/ryanstevens/Documents/github/torch_practice/data/images/simple_encoder_decoder_graph.png')

Above we have a graphical representation of a simple structure, we encode (compress) data using the function $g(x)$, and then decode (uncompress) data using the inverse, $f(x)=g^{-1}(x)$. To arrive at these transformations, we can define any mappings we desire. We don't have to use simple linear transformations, $g(x)$ and $f(x)$ can be any mapping we want (i.e. a complex neural network). 

The goal of autoencoders is to find the mappings to minimize reconstruction loss. Formally, we try to solve the following problem:

$$
\begin{aligned}
\min_{(\theta_{x}, \theta_{z})} &\Big\| \bold{x} - f_{\theta_x}(g_{\theta_{z}}(\bold{x})) \Big\|  \\
&\text{s.t.} \\
&g_{\theta_{z}} : \mathbb{R}^{n} \to \mathbb{R}^{m} &(\text{encoder}) \\
&f_{\theta_{x}} : \mathbb{R}^{m} \to \mathbb{R}^{n} &(\text{decoder})
\end{aligned}
$$

## Connection to Principal Component Analysis

It's useful at this point to make a connection to PCA. PCA is a common dimensionality reduction technique. The intuitive idea behind PCA is that the "directions" of high variation are good candidates for basis to project a high-dimensional vector down to. 

A linear PCA is one class of encoder-decoder functions (linear functions), with a specific reconstruction loss (l2-norm). Specifically, the following functional form:

$$ 
\begin{align*}
f(x) = Wx \;&;\; g(z) = Vz \\
\min_{W, V} \frac{1}{N} \sum_{k} &\lVert x^{k} - g(f(x^{k})) \rVert 

\end{align*}
$$

All our general encoder-decoder architecture allows us to do is to do non-linear mappings for $g$ and $f$ and potentially use different reconstruction losses. 

## Notation

At this point, let's clarify notation. For any matrix, $y \in \mathbb{R}^{m\text{ x }n}$, we do the following:

* Superscripts define the unit: $y^{k}$ is the kth unit, $k \in (1...K)$
* Subscripts define the row, column within a unit: $y^{k}_{i, j}$ is the $i$th row and $j$th column for unit $k$, $(i \in (1...I), j \in (1...J))$

Our problem is focused on images, thus each image, indexed by $k$, we define as two-dimensional matrix, $\bold{x}^{k} \in \mathbb{R}^{I\text{ x }J}$.

# Toy Problem: Identifying Anomalous Images

We will be investigating a toy problem to show the different approaches in code. We will use images of handwritten digits from the MNIST dataset. We choose this dataset because it's easy to download from pytorch. Below we show an example of a typical image:

## What is our problem?

Our goal is to identify "anomalous images", we will define "anomalous" more precisely below. We have a two-step process for "anomaly detection":

1. Train an autoencoder $(\hat{\theta_{x}}, \hat{\theta_{z}})$ on a training dataset, minimizing reconstruction loss.
2. Pass through a test image, $\bold{x}^{k}$ to generate a reconstructed image $\tilde{\bold{x}}^{k}$
3. Define some function, $A$ and threshold, $t$ of the test image and it's reconstruction loss to determine anomalies: $A(\bold{x}^{k}, \tilde{\bold{x}}^{k}) \geq t$. 

## The Data

### Raw Data

Our dataset consists of 2 dimensional images in gray scale. An image represents a digit, where a label `l^{k} = I` is an integer, `I`, between 0 and 9. 

To use this for `anomaly detection`, we split our data into two groups based on Integer. 
* `inliers`: integers we train our autoencoder on. Note, we further split this group into a `train` and `test` set.
* `outliers`: integers we want our autoencoder to identify as anomalous. This group doesn't exist in our `train` set, and therfore only exists in our test set

Basically, we train our autoencoder on "normal" images (`train - inliers`). We then pass through a series of `test` images, which are a combination of `inliers` and `outliers`.

To keep this document terse, I will not present the code on generating our `train` and `test` sets. The code and dataset are available in this public repo. 

### Data Preprocessing 

we start with images, defined as 2 dimensional matrices $\bold{x} \in \mathbb{R}^{2}$. We do two preprocessing steps:
* Convert each pixel to value between 0 and 1
* Vertically stack columns to transform two-dimensional $x \in \mathbb{R}^{I\text{ x }J}$ to one-dimension $x \in \mathbb{R}^{IJ}$

There are two notes with our approach:
* We will still keep the subscript indexing the same $x_{i, j}$ even though we've vertically stacked our observations
* Vertical stacking was a simplification: instead of vertically stacking we could do a convolutional NN approach, we choose not to for this exposition. 

## Training our AutoEncoder

The approach used will be to do unsupervised training of our autoencoder. Given that we converted each our pixels to values between 0 and 1, we will use a binary cross-entropy (logloss) objective function:

$$
\begin{aligned}
\sum_{k} \sum_{i, j} log(\tilde{x}^{k}_{i, j})*x^{k}_{i, j} + log(1-\tilde{x}^{k}_{i, j})*(1-x^{k}_{i, j})
\end{aligned}
$$


# Autoencoders Tutorial

At work, we've been starting to explore different approaches to anomaly detection. I'm creating a series of posts to clarify my understanding of autoencoders. A lot of this material will be fairly basic for many readers, but I want to start with the basics and build up. 

## Why do we need autoencoders? 

The key idea is lossless compression. Below shows the process of compressing and uncompressing a set of data points from $\mathbb{R}^{2} \to \mathbb{R}^{1}$ and back to $\mathbb{R}^{1} \to \mathbb{R}^{2}$. In the example, we have three sets of data points:
* Input: $\bold{x} \in \mathbb{R}^{2}$
* Compressed by map $g_{\theta_{z}}$: $\bold{z} \in \mathbb{R}^{1}$
* Reconstructed by map $f_{\theta_{x}}$: $\bold{\tilde{x}} \in \mathbb{R}^{2}$


Above we have a graphical representation of transformations. To arrive at these transformations, we can define any mappings we desire. An example could be a neural network, that has this structure:

The goal of autoencoders is to find the mappings to minimize reconstruction loss. Formally, we try to solve the following problem:

$$
\begin{aligned}
\min_{(\theta_{x}, \theta_{z})} &\Big\| \bold{x} - f_{\theta_x}(g_{\theta_{z}}(\bold{x})) \Big\|  \\
&\text{s.t.} \\
&g_{\theta_{z}} : \mathbb{R}^{n} \to \mathbb{R}^{m} &(\text{encoder}) \\
&f_{\theta_{x}} : \mathbb{R}^{m} \to \mathbb{R}^{n} &(\text{decoder})
\end{aligned}
$$

## Defining notation

At this point, let's clarify notation. For any matrix, $y \in \mathbb{R}^{m\text{ x }n}$, we do the following:

* Superscripts define the unit: $y^{k}$ is the kth unit, $k \in (1...K)$
* Subscripts define the row, column within a unit: $y^{k}_{i, j}$ is the $i$th row and $j$th column for unit $k$, $(i \in (1...I), j \in (1...J))$

Our problem is focused on images, thus each image, indexed by $k$, we define as two-dimensional matrix, $\bold{x}^{k} \in \mathbb{R}^{I\text{ x }J}$.

# Toy Problem: Identifying Anomalous Images

We will be investigating a toy problem to show the different approaches in code. We will use images of handwritten digits from the MNIST dataset. We choose this dataset because it's easy to download from pytorch. 

## What is our problem?

Our goal is to identify "anomalous images", we will define "anomalous" more precisely below. We have a two-step process for "anomaly detection":

1. Train an autoencoder $(\hat{\theta_{x}}, \hat{\theta_{z}})$ on a training dataset, minimizing reconstruction loss.
2. Pass through a test image, $\bold{x}^{k}$ to generate a reconstructed image $\tilde{\bold{x}}^{k}$
3. Define some function, $A$ and threshold, $t$ of the test image and it's reconstruction loss to determine anomalies: $A(\bold{x}^{k}, \tilde{\bold{x}}^{k}) \geq t$. 

## What is our raw data?

Our dataset consists of 2 dimensional images in gray scale. An image represents a digit, where a label `l^{k} = I` is an integer, `I`, between 0 and 9. 

To use this for `anomaly detection`, we split our data into two groups based on Integer. 
* `inliers`: integers we train our autoencoder on. Note, we further split this group into a `train` and `test` set.
* `outliers`: integers we want our autoencoder to identify as anomalous. This group doesn't exist in our `train` set, and therfore only exists in our test set

Basically, we train our autoencoder on "normal" images (`train - inliers`). We then pass through a series of `test` images, which are a combination of `inliers` and `outliers`.

To keep this document terse, I will not present the code on generating our `train` and `test` sets. The code and dataset are available in this public repo. 

## What is our data preprocessing?

we start with images, defined as 2 dimensional matrices $\bold{x} \in \mathbb{R}^{2}$. We do two preprocessing steps:
* Convert each pixel to value between 0 and 1
* Vertically stack columns to transform two-dimensional $x \in \mathbb{R}^{I\text{ x }J}$ to one-dimension $x \in \mathbb{R}^{IJ}$

There are two notes with our approach:
* We will still keep the subscript indexing the same $x_{i, j}$ even though we've vertically stacked our observations
* Vertical stacking was a simplification: instead of vertically stacking we could do a convolutional NN approach, we choose not to for this exposition. 

## What is our approcah?

The approach used will be to do unsupervised training of our autoencoder. Given that we converted each our pixels to values between 0 and 1, we will use a binary cross-entropy (logloss) objective function:

$$
\begin{aligned}
\sum_{k} \sum_{i, j} log(\tilde{x}^{k}_{i, j})*x^{k}_{i, j} + log(1-\tilde{x}^{k}_{i, j})*(1-x^{k}_{i, j})
\end{aligned}
$$

# Exploring Autoencoders

## Loading data

We will load data that we will use throughout this tutorial.


# Exploring Autoencoders

## Exploring Bottleneck Layer

The key choice for the autoencoder, the size of the latent dimension (or bottleneck layer). In our terminology, it's the dimension of the target space ($m$) of our encoder, $g_{\theta_{z}} : \mathbb{R}^{n} \to \mathbb{R}^{m}$.

Below shows an example vanilla architecture that has a single hidden layer that maps $\mathbb{R}^{8} \to \mathbb{R}^{4}$. Note, we only have one single hidden unit. Like any NN we can increase the depth by adding more hidden units.

![](/Users/ryanstevens/Documents/github/torch_practice/data/images/vanilla_autoencoder.png)


The main choice with a vanilla autoencoder is the size of the bottleneck layer (i.e. $m$). To build intuition, the key is to think about two polar opposite scenarios:
* `dim(z) = 1`: this reduces all our dimension a single scalar. This highly compresses the data, which would lead to poor generalization error if `dim(x)` is large
* `dim(z) = dim(x)`: there is no compression, thus the training error will be 0, as it simply "memorizes" the data, thus poor generationalization error.

In fact, we could roughly think about generalization error as some convex function between these two poles:

![Generalization Error and Latent Space Size](/Users/ryanstevens/Documents/github/torch_practice/notebooks/blog_posts/images/generalization_error.png)

### Code

We now define this in code. We create a class `VanillaEncoderDecoder` to make it easier to define our vanilla architecture, specifically looping through different bottleneck layer sizes. 

Let's print out the architecture for one example, just so we can see the full architecture. We use `ReLU` units, `dropout(p=0.5)`, and a sigmoid activation unit for the final head layer.

In [ ]:
# Load modules 
import torch.nn as nn
from src.helpers import models
from torchview import draw_graph

# Example vanilla autoencoder
vanilla_model_example = models.StaticEncoderDecoder(
    input_dim=28*28,
    latent_dim=4,
    head_layer=nn.Sigmoid()
)

# Plot graph
model_graph = draw_graph(vanilla_model_example, input_size=(1, 28*28))
model_graph.visual_graph

In [ ]:
%markdown

## Examining Bottleneck Layer

We will now explore the impact of changing the size of the bottleneck layer. We will train different `VanillaEncoderDecoder` varying only the `latent_dim` parameter. We will explore some interesting statistics and plots around this. Note, I built a helper function `data.load_and_preprocess_data()` to load, preprocess and split our data into `train`, `val`, `test` sets. It's all available in this part of our repo:...


# References

* [Pytorch Tutorials of AutoEncoder Papers](https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb)

In [4]:
import sys; sys.path.insert(0, '/Users/ryanstevens/Documents/github/torch_practice/')
from src.helpers import data
from src.helpers import models, training
import torch.nn as nn
import numpy as np 
import torch
from importlib import reload
from pathlib import Path
reload(models)
reload(training)
reload(data)

# Set parameters
INPUT_DIM=28**2
LOSS_FN=nn.CrossEntropyLoss()
EPOCHS=200
LR=1e-4

losses=[]
for batch_size in [
    4, 
    16, 
    32]:
    
    # Load dataset 
    train, val, test = data.load_and_preprocess_data(debug=False, batch_size=batch_size)

    # Save the first batch from the validation dataset
    path_to_val_batch_prediction_folder = f'/Users/ryanstevens/Documents/github/torch_practice/data/batch_predictions/validation_data/batchSize={batch_size}'
    Path(path_to_val_batch_prediction_folder).mkdir(parents=True, exist_ok=True)
    first_batch_X_val, first_batch_y_val = next(iter(val))

    with open(Path(path_to_val_batch_prediction_folder) / 'y.npy', 'wb') as f:
        np.save(f, first_batch_y_val.numpy())
    
    with open(Path(path_to_val_batch_prediction_folder) / 'X.npy', 'wb') as f:
        np.save(f, first_batch_X_val.numpy())

    for latent_dim, compression_per_layer in [
        # (1, 16), 
        # (64, 2), 
        (128, 2),
        (256, 2), 
        (28*28, 2)
    ]:
        if latent_dim == 1 and batch_size == 16:
            continue 

        # Create path to batch predictions
        path_to_batch_prediction_folder = f'/Users/ryanstevens/Documents/github/torch_practice/data/batch_predictions/static/latentDim={latent_dim}_batchSize={batch_size}'
        path_to_early_stopping_folder=f'/Users/ryanstevens/Documents/github/torch_practice/data/models/static/latentDim={latent_dim}_batchSize={batch_size}'
        Path(path_to_batch_prediction_folder).mkdir(parents=True, exist_ok=True)
        Path(path_to_early_stopping_folder).mkdir(parents=True, exist_ok=True)

        # Create encoder-decoder
        encoder_decoder_model = models.StaticEncoderDecoder(
            input_dim=INPUT_DIM,
            latent_dim=latent_dim,
            head_layer=nn.Sigmoid(),
            compression_per_layer=compression_per_layer,
        )

        # Initialize early stopping
        early_stopping=training.EarlyStopping(
            patience=5, 
            verbose=True, 
            path=path_to_early_stopping_folder
        )

        # Initialize model training
        model_trainer=training.ModelTrainer(
            model=encoder_decoder_model,
            optimizer=torch.optim.Adam(encoder_decoder_model.parameters(), lr=LR),
            loss_fn=LOSS_FN,
            train_loader=train,
            val_loader=val,
            epochs=EPOCHS,
            early_stopping=early_stopping,
            path_to_batch_prediction_folder=path_to_batch_prediction_folder
        )

        # Train model
        model_trainer.train()

        # Persist training and validation losses
        losses.append(
            {
                'train_losses': model_trainer.train_losses,
                'val_losses': model_trainer.val_losses,
                'test_losses': [LOSS_FN(model_trainer.best_model(X), X) for _, (X, _) in enumerate(test)],
                'latent_dim': latent_dim
            }
        )

train loss: 646.289490  [    1/ 1184]
train loss: 605.166626  [  101/ 1184]
train loss: 687.562195  [  201/ 1184]
train loss: 614.589233  [  301/ 1184]
train loss: 621.595764  [  401/ 1184]
train loss: 623.302063  [  501/ 1184]
train loss: 603.255981  [  601/ 1184]
train loss: 553.200500  [  701/ 1184]
train loss: 650.619873  [  801/ 1184]
train loss: 630.767456  [  901/ 1184]
train loss: 602.992310  [ 1001/ 1184]
train loss: 644.702332  [ 1101/ 1184]


2023-11-26 18:56:10.387 | INFO     | src.helpers.training:_run_single_loop:75 - Saving a single batch of data
2023-11-26 18:56:10.389 | INFO     | src.helpers.training:_run_single_loop:76 - Numpy shape: (32, 784)
2023-11-26 18:56:12.385 | INFO     | src.helpers.training:_run_single_loop:87 - Size of batch validation: 395
2023-11-26 18:56:12.386 | INFO     | src.helpers.training:_run_single_loop:88 - First 5 rows of batch validation loss: [630.306884765625, 655.218505859375, 562.4224853515625, 611.0111083984375, 590.0322875976562]
2023-11-26 18:56:12.387 | INFO     | src.helpers.training:__call__:153 - best_score is None, saving best score...
2023-11-26 18:56:12.388 | INFO     | src.helpers.training:save_checkpoint:171 - Validation loss decreased (inf --> 613.896164).  Saving model ...


train loss: 653.765137  [    1/ 1184]
train loss: 656.082642  [  101/ 1184]
train loss: 638.029602  [  201/ 1184]
train loss: 589.501099  [  301/ 1184]
train loss: 730.439758  [  401/ 1184]
train loss: 620.834656  [  501/ 1184]
train loss: 614.702271  [  601/ 1184]
train loss: 617.128601  [  701/ 1184]
train loss: 678.677673  [  801/ 1184]
train loss: 554.435059  [  901/ 1184]
train loss: 629.238525  [ 1001/ 1184]
train loss: 636.730042  [ 1101/ 1184]


2023-11-26 18:56:23.199 | INFO     | src.helpers.training:_run_single_loop:75 - Saving a single batch of data
2023-11-26 18:56:23.200 | INFO     | src.helpers.training:_run_single_loop:76 - Numpy shape: (32, 784)
2023-11-26 18:56:25.166 | INFO     | src.helpers.training:_run_single_loop:87 - Size of batch validation: 395
2023-11-26 18:56:25.167 | INFO     | src.helpers.training:_run_single_loop:88 - First 5 rows of batch validation loss: [626.5169677734375, 651.0533447265625, 558.7174072265625, 607.8361206054688, 585.4027709960938]
2023-11-26 18:56:25.167 | INFO     | src.helpers.training:__call__:163 - current score -609.5198641620104 is higher than -613.8961639017998, setting new best score to -609.5198641620104
2023-11-26 18:56:25.168 | INFO     | src.helpers.training:save_checkpoint:171 - Validation loss decreased (613.896164 --> 609.519864).  Saving model ...


train loss: 570.642456  [    1/ 1184]
train loss: 601.864319  [  101/ 1184]
train loss: 682.532104  [  201/ 1184]
train loss: 594.178528  [  301/ 1184]
train loss: 580.658875  [  401/ 1184]
train loss: 527.713989  [  501/ 1184]
train loss: 591.174072  [  601/ 1184]
train loss: 593.221436  [  701/ 1184]
train loss: 693.059387  [  801/ 1184]
train loss: 579.752014  [  901/ 1184]
train loss: 658.743958  [ 1001/ 1184]
train loss: 667.921082  [ 1101/ 1184]


...

train loss: 573.447876  [    1/ 1184]
train loss: 586.446960  [  101/ 1184]
train loss: 626.090210  [  201/ 1184]
train loss: 547.929382  [  301/ 1184]
train loss: 605.107910  [  401/ 1184]
train loss: 568.391724  [  501/ 1184]
train loss: 586.916870  [  601/ 1184]
train loss: 602.143982  [  701/ 1184]
train loss: 623.383789  [  801/ 1184]
train loss: 528.793701  [  901/ 1184]
train loss: 601.283691  [ 1001/ 1184]
train loss: 552.064026  [ 1101/ 1184]


2023-11-26 21:23:18.730 | INFO     | src.helpers.training:_run_single_loop:75 - Saving a single batch of data
2023-11-26 21:23:18.731 | INFO     | src.helpers.training:_run_single_loop:76 - Numpy shape: (32, 784)
2023-11-26 21:23:20.266 | INFO     | src.helpers.training:_run_single_loop:87 - Size of batch validation: 395
2023-11-26 21:23:20.267 | INFO     | src.helpers.training:_run_single_loop:88 - First 5 rows of batch validation loss: [607.56494140625, 631.1932373046875, 539.860107421875, 586.4642333984375, 568.3346557617188]
2023-11-26 21:23:20.268 | INFO     | src.helpers.training:__call__:157 - current score -591.8049815503857 is lower than -591.8034766861155, incrementing patience counter
2023-11-26 21:23:20.269 | INFO     | src.helpers.training:__call__:159 - EarlyStopping counter: 2 out of 5


train loss: 626.283752  [    1/ 1184]
train loss: 583.819336  [  101/ 1184]
train loss: 579.230103  [  201/ 1184]
train loss: 578.072021  [  301/ 1184]
train loss: 624.515381  [  401/ 1184]
train loss: 638.566223  [  501/ 1184]
train loss: 598.621582  [  601/ 1184]
train loss: 627.205933  [  701/ 1184]
train loss: 607.109192  [  801/ 1184]
train loss: 594.777466  [  901/ 1184]
train loss: 574.675720  [ 1001/ 1184]
train loss: 554.625122  [ 1101/ 1184]


2023-11-26 21:23:30.710 | INFO     | src.helpers.training:_run_single_loop:75 - Saving a single batch of data
2023-11-26 21:23:30.711 | INFO     | src.helpers.training:_run_single_loop:76 - Numpy shape: (32, 784)
2023-11-26 21:23:32.369 | INFO     | src.helpers.training:_run_single_loop:87 - Size of batch validation: 395
2023-11-26 21:23:32.370 | INFO     | src.helpers.training:_run_single_loop:88 - First 5 rows of batch validation loss: [607.6260986328125, 631.1261596679688, 539.846923828125, 586.5072021484375, 568.3600463867188]
2023-11-26 21:23:32.371 | INFO     | src.helpers.training:__call__:163 - current score -591.7961561758307 is higher than -591.8034766861155, setting new best score to -591.7961561758307
2023-11-26 21:23:32.372 | INFO     | src.helpers.training:save_checkpoint:171 - Validation loss decreased (591.803477 --> 591.796156).  Saving model ...
2023-11-26 21:23:32.385 | INFO     | src.helpers.training:train:40 - Early stopping is set: loading best model from /Users/r

ValueError: optimizer got an empty parameter list

In [ ]:
%markdown

## Analyzing model performance

### Model Losses

We first explore the relationship between the bottleneck layer and the train, validation, and test losses. We see the general pattern discussed earlier.

Also, it's interesting to have a look at some reconstructed images, we persisted images for the validation set across the batches / epochs. Below, we've created a grid of gifs. The columns are the size of the bottleneck layer, the rows are the batch sizes. 

## Anomaly Detection
We can now use this simple autoencoder for anomaly detection. Let's simply plot 